<a href="https://colab.research.google.com/github/07tia/airbnb/blob/main/Neo4j_in_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install & Connect**

In [ ]:
# Step 1: Install the Neo4j Python Driver
!pip install neo4j

# Step 2: Connect to the AuraDB instance
from neo4j import GraphDatabase

# Use the URI shown in your screenshot
URI = "neo4j+s://c257d2c3.databases.neo4j.io"

# Replace with the username and password shown earlier
AUTH = ("neo4j", "Ujh9-qUPDdgG2zO4Y6x3UEbCF0gRiUqN1PGdZm7GxkQ")

# Step 3: Initialize the driver and verify connection
driver = GraphDatabase.driver(URI, auth=AUTH)

def test_connection(tx):
    return tx.run("RETURN 'Connected to Neo4j AuraDB!' AS message").single()["message"]

with driver.session() as session:
    print(session.execute_read(test_connection))


Connected to Neo4j AuraDB!


# **Load CSV**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import the pandas library with the alias 'pd'
import pandas as pd

file_path = '/content/drive/MyDrive/BUDT 737: Group Project/Final Final Project Airbnb/combined_listings.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-417dfadbd000>:8: DtypeWarning: Columns (28,29,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
df.head()
df.columns


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

# **Define Graph Creation Logic**

In [ ]:
def create_airbnb_graph(tx, row):
    tx.run("""
        MERGE (l:Listing {id: $listing_id})
        SET l.price = $price,
            l.neighborhood = $neighborhood,
            l.room_type = $room_type,
            l.property_type = $property_type

        MERGE (h:Host {id: $host_id})
        MERGE (h)-[:OWNS]->(l)

        MERGE (n:Neighborhood {name: $neighborhood})
        MERGE (l)-[:LOCATED_IN]->(n)
    """,
    listing_id=row["id"],
    host_id=row["host_id"],
    price=row["price"],
    neighborhood=row["neighbourhood_cleansed"],
    room_type=row["room_type"],
    property_type=row["property_type"])


# **Prepare Data and Execute Transactions**

In [ ]:
columns = ["id", "host_id", "price", "neighbourhood_cleansed", "room_type", "property_type"]
df_small = df[columns].dropna().head(500)

with driver.session() as session:
    for _, row in df_small.iterrows():
        session.write_transaction(create_airbnb_graph, row)


<ipython-input-7-d7607de3423a>:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_airbnb_graph, row)


# **Queries**

# **Business Questions**

**#1 What makes an Airbnb listing high-priced and why?**

High-priced listings are mostly entire homes in popular neighborhoods like Southwest Waterfront and West End.

These listings are often owned by multi-property hosts.

Key factors: location, room type, and host behavior.



*Shows listings over $100 with related host and neighborhood nodes:*

In [ ]:
# High-priced listings with their host and neighborhood (Visualizable)
with driver.session() as session:
    result = session.run("""
        MATCH (h:Host)-[:OWNS]->(l:Listing)-[:LOCATED_IN]->(n:Neighborhood)
        WHERE toFloat(l.price) > 100
        RETURN h.id AS host_id, l.id AS listing_id, l.price, n.name AS neighborhood
        LIMIT 50
    """)
    for record in result:
        print(record)


<Record host_id=5326343 listing_id=10617103 l.price='385.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record host_id=59087346 listing_id=11303521 l.price='107.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record host_id=11771833 listing_id=12623775 l.price='499.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record host_id=71002747 listing_id=13865075 l.price='221.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record host_id=38345667 listing_id=7319641 l.price='276.0' neighborhood='Hawthorne, Barnaby Woods, Chevy Chase'>
<Record host_id=46630199 listing_id=8967165 l.price='133.0' neighborhood='Hawthorne, Barnaby Woods, Chevy Chase'>
<Record host_id=2534048 listing_id=4565447 l.price='246.0' neighborhood='Ivy City, Arboretum, Trinidad, Carver Langston'>
<Record host_id=4176601 listing_id=1825

**2. Are specific neighborhoods saturated with listings?**

Capitol Hill and Union Station top the list with 50+ listings each, indicating dense Airbnb presence.

Other areas like Columbia Heights and Dupont Circle also show high listing volumes (30–40).

Graph in Neo4j shows dense clusters of listings in a few neighborhoods.

Suggests oversaturation in certain areas, impacting local housing.



In [ ]:
# Count of listings per neighborhood
with driver.session() as session:
    result = session.run("""
        MATCH (l:Listing)-[:LOCATED_IN]->(n:Neighborhood)
        RETURN n.name AS neighborhood, COUNT(l) AS listing_count
        ORDER BY listing_count DESC
        LIMIT 50
    """)
    for record in result:
        print(record)


<Record neighborhood='Capitol Hill, Lincoln Park' listing_count=56>
<Record neighborhood='Union Station, Stanton Park, Kingman Park' listing_count=53>
<Record neighborhood='Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View' listing_count=43>
<Record neighborhood='Brightwood Park, Crestwood, Petworth' listing_count=32>
<Record neighborhood='Dupont Circle, Connecticut Avenue/K Street' listing_count=30>
<Record neighborhood='Edgewood, Bloomingdale, Truxton Circle, Eckington' listing_count=27>
<Record neighborhood='Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street' listing_count=25>
<Record neighborhood='Shaw, Logan Circle' listing_count=24>
<Record neighborhood='Kalorama Heights, Adams Morgan, Lanier Heights' listing_count=23>
<Record neighborhood='Georgetown, Burleith/Hillandale' listing_count=20>
<Record neighborhood='Takoma, Brightwood, Manor Park' listing_count=18>
<Record neighborhood='Howard University, Le Droit Park, Cardozo/Shaw' listing_count=

Visualization

In [ ]:
# Neighborhoods and their listings
with driver.session() as session:
    result = session.run("""
        MATCH (n:Neighborhood)<-[:LOCATED_IN]-(l:Listing)
        RETURN n.name AS neighborhood, l.id AS listing_id
        LIMIT 50
    """)
    for record in result:
        print(record)


<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=1205516>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=4231934>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=6334405>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=6404202>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=7714880>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=10617103>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=11303521>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' listing_id=12623775>
<Record neighborhood='Southwest Employment Ar

**3. Do hosts with multiple properties impact pricing or availability?**

Hosts with the most listings (up to 15) don’t always charge the most one with 15 listings averages just $35.13.

Some mid-tier hosts (5–6 listings) charge over $300 on average.

This shows that volume doesn't equal premium pricing — high-priced listings are often held by selective, not high-volume, hosts.

Useful for platform policy (e.g., flagging large-scale operators) and host strategy (e.g., balancing volume vs. price).

In [ ]:
# Hosts with multiple listings and their average listing price
with driver.session() as session:
    result = session.run("""
        MATCH (h:Host)-[:OWNS]->(l:Listing)
        WHERE l.price IS NOT NULL
        WITH h, COUNT(l) AS listing_count, AVG(toFloat(l.price)) AS avg_price
        RETURN h.id AS host_id, listing_count, avg_price
        ORDER BY listing_count DESC
        LIMIT 10
    """)
    for record in result:
        print(record)


<Record host_id=315148 listing_count=15 avg_price=35.13333333333333>
<Record host_id=41047431 listing_count=7 avg_price=144.14285714285714>
<Record host_id=9419684 listing_count=6 avg_price=323.5>
<Record host_id=5487930 listing_count=6 avg_price=132.66666666666669>
<Record host_id=46630199 listing_count=5 avg_price=269.2>
<Record host_id=17633 listing_count=5 avg_price=149.4>
<Record host_id=14692970 listing_count=5 avg_price=133.2>
<Record host_id=30283594 listing_count=5 avg_price=305.6>
<Record host_id=41656215 listing_count=5 avg_price=225.0>
<Record host_id=16644446 listing_count=5 avg_price=507.6>


In [ ]:
# Visualize top commercial hosts
with driver.session() as session:
    result = session.run("""
        MATCH (h:Host)-[:OWNS]->(l:Listing)
        WITH h, COUNT(l) AS count
        WHERE count > 5
        MATCH (h)-[:OWNS]->(l)
        RETURN h.id AS host_id, l.id AS listing_id, l.price
        LIMIT 50
    """)
    for record in result:
        print(record)


<Record host_id=9419684 listing_id=1864715 l.price='234.0'>
<Record host_id=9419684 listing_id=1867513 l.price='413.0'>
<Record host_id=9419684 listing_id=2371799 l.price='413.0'>
<Record host_id=9419684 listing_id=4092017 l.price='234.0'>
<Record host_id=9419684 listing_id=4447266 l.price='234.0'>
<Record host_id=9419684 listing_id=4452984 l.price='413.0'>
<Record host_id=5487930 listing_id=2964999 l.price='145.0'>
<Record host_id=5487930 listing_id=4022565 l.price='135.0'>
<Record host_id=5487930 listing_id=4218583 l.price='142.0'>
<Record host_id=5487930 listing_id=4343926 l.price='108.0'>
<Record host_id=5487930 listing_id=4411488 l.price='122.0'>
<Record host_id=5487930 listing_id=11848786 l.price='144.0'>
<Record host_id=41047431 listing_id=7800582 l.price='162.0'>
<Record host_id=41047431 listing_id=7801335 l.price='128.0'>
<Record host_id=41047431 listing_id=8645356 l.price='135.0'>
<Record host_id=41047431 listing_id=8645554 l.price='128.0'>
<Record host_id=41047431 listing_id

**4. How can these insights guide better decisions for cities and hosts?**


The top 10 most expensive neighborhoods have average prices ranging from $154 to $292.

“Near Southeast, Navy Yard” and “Downtown, Chinatown, Penn Quarters” stand out with the highest average prices, over $280.

Many high-priced areas (“Cathedral Heights”, “Historic Anacostia”) are high-demand zones, indicating premium listing potential.

This data helps cities focus regulatory or tourism resources in higher-value zones, and helps hosts assess where listings can charge more.



In [ ]:
# Average price per neighborhood
with driver.session() as session:
    result = session.run("""
        MATCH (l:Listing)-[:LOCATED_IN]->(n:Neighborhood)
        WHERE l.price IS NOT NULL
        RETURN n.name AS neighborhood, AVG(toFloat(l.price)) AS avg_price
        ORDER BY avg_price DESC
        LIMIT 10
    """)
    for record in result:
        print(record)


<Record neighborhood='Near Southeast, Navy Yard' avg_price=292.0>
<Record neighborhood='Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street' avg_price=283.12>
<Record neighborhood='Cathedral Heights, McLean Gardens, Glover Park' avg_price=187.27272727272725>
<Record neighborhood='Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace' avg_price=181.44444444444446>
<Record neighborhood='Historic Anacostia' avg_price=172.8>
<Record neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point' avg_price=172.66666666666666>
<Record neighborhood='Kalorama Heights, Adams Morgan, Lanier Heights' avg_price=170.43478260869566>
<Record neighborhood='Union Station, Stanton Park, Kingman Park' avg_price=168.9811320754718>
<Record neighborhood='Capitol Hill, Lincoln Park' avg_price=159.30357142857144>
<Record neighborhood='Georgetown, Burleith/Hillandale' avg_price=154.70000000000002>


In [ ]:
# Listings over $150 in expensive neighborhoods
with driver.session() as session:
    result = session.run("""
        MATCH (l:Listing)-[:LOCATED_IN]->(n:Neighborhood)
        WHERE toFloat(l.price) > 150
        RETURN l.id AS listing_id, l.price, n.name AS neighborhood
        LIMIT 50
    """)
    for record in result:
        print(record)


<Record listing_id=10617103 l.price='385.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record listing_id=12623775 l.price='499.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record listing_id=13865075 l.price='221.0' neighborhood='Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point'>
<Record listing_id=7319641 l.price='276.0' neighborhood='Hawthorne, Barnaby Woods, Chevy Chase'>
<Record listing_id=4565447 l.price='246.0' neighborhood='Ivy City, Arboretum, Trinidad, Carver Langston'>
<Record listing_id=11176974 l.price='163.0' neighborhood='West End, Foggy Bottom, GWU'>
<Record listing_id=11866275 l.price='166.0' neighborhood='West End, Foggy Bottom, GWU'>
<Record listing_id=11868093 l.price='157.0' neighborhood='West End, Foggy Bottom, GWU'>
<Record listing_id=11868625 l.price='167.0' neighborhood='West End, Foggy Bottom, GWU'>
<Record listing_id=6491631 l.price='4